## PATTERN 3: PROMPT TEMPLATE ROUTING

### Routes queries to different **prompt templates** based on the desired response style:

- **technical**: Coding, engineering, APIs  
- **creative**: Writing, storytelling, brainstorming  
- **educational**: Learning, tutorials, explanations  
- **analytical**: Comparisons, analysis, decision-making  
- **conversational**: Casual chat, simple Q&A  

```
Diagram:
    Query → Router → Prompt 1 (Technical) ─┐
                  → Prompt 2 (Creative)  ──┼── → LLM → Response
                  → Prompt 3 (Educational)─┘
```


All routes go to the **same LLM**, but use **different system prompts**.


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from typing import Literal

# Import from config
from config import llm, PROMPT_TEMPLATES


NameError: name 'OPENAI_API_KEY' is not defined

In [3]:
class PromptRoute(BaseModel):
    """Structured output for prompt routing."""
    prompt_type: Literal["technical", "creative", "educational", "analytical", "conversational"] = Field(
        description="The prompt template style to use"
    )
    reasoning: str = Field(description="Why this prompt style was chosen")
    confidence: float = Field(ge=0.0, le=1.0, description="Confidence score")


In [ ]:
def create_router():
    """Create the prompt template router."""
    system = """You are an expert at choosing the right response style.

PROMPT TEMPLATES:
- technical: Coding, engineering, APIs, debugging, system design
- creative: Stories, poems, marketing copy, brainstorming, naming ideas
- educational: "Explain X", "How does Y work?", teaching, tutorials
- analytical: "Compare A vs B", analysis, pros/cons, decision-making
- conversational: Casual chat, simple Q&A, greetings, personal advice

ROUTING RULES:
- "How do I implement..." → technical
- "Debug this code..." → technical
- "Write a story/poem about..." → creative
- "Give me creative names for..." → creative
- "Explain X in simple terms..." → educational
- "Teach me about..." → educational
- "Compare X and Y..." → analytical
- "What are the pros and cons..." → analytical
- "Hey, how are you?" → conversational
- "Thanks!" → conversational

Choose the template that matches the query's desired response style."""

    router_prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "{question}")
    ])
    
    structured_llm = llm.with_structured_output(PromptRoute)
    return router_prompt | structured_llm


router = create_router()

In [ ]:
def query_rag(question: str):
    """
    Route and answer a question using prompt template routing.
    
    Args:
        question: User question
    
    Returns:
        Dict with prompt_type, confidence, reasoning, answer
    """
    print(f"\n{'='*80}")
    print(f" QUESTION: {question}")
    print(f"{'='*80}")
    
    # Step 1: Route the query
    route = router.invoke({"question": question})
    print(f" Routing to: {route.prompt_type} prompt (confidence: {route.confidence:.2f})")
    print(f"   Reasoning: {route.reasoning}")
    
    # Step 2: Get the appropriate prompt template (IF-ELSE LOGIC)
    if route.prompt_type in PROMPT_TEMPLATES:
        system_prompt = PROMPT_TEMPLATES[route.prompt_type]
    else:
        system_prompt = PROMPT_TEMPLATES["conversational"]
    
    # Step 3: Generate response with selected prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{question}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"question": question})
    
    print(f"\n ANSWER ({route.prompt_type} style):\n{'-'*80}\n{answer}\n{'-'*80}\n")
    
    return {
        "prompt_type": route.prompt_type,
        "confidence": route.confidence,
        "reasoning": route.reasoning,
        "answer": answer
    }


In [ ]:
def compare_all_prompts(question: str):
    """
    Generate responses using ALL prompt templates for comparison.
    Useful for demonstrating how different prompts affect output.
    """
    print(f"\n{'='*80}")
    print(f" COMPARING ALL PROMPTS: {question}")
    print(f"{'='*80}")
    
    results = {}
    
    for prompt_type, system_prompt in PROMPT_TEMPLATES.items():
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "{question}")
        ])
        
        chain = prompt | llm | StrOutputParser()
        answer = chain.invoke({"question": question})
        
        results[prompt_type] = answer
        
        print(f"\n {prompt_type.upper()}:\n{'-'*40}")
        print(answer[:300] + "..." if len(answer) > 300 else answer)
    
    return results


In [ ]:
# Test queries
test_questions = [
        ("How do I implement a REST API in Python?", "technical"),
        ("Write a short poem about coding", "creative"),
        ("Explain neural networks to a beginner", "educational"),
        ("Compare PostgreSQL vs MongoDB", "analytical"),
        ("Hey, how are you today?", "conversational"),
    ]
    
for question, expected in test_questions:
    result = query_rag(question)
    match = "OK" if result["prompt_type"] == expected else "X"
        print(f"{match} Expected: {expected} | Got: {result['prompt_type']}\n")
    
# Demo: Compare same question with different prompts
print("\n" + "=" * 80)
print("DEMO: Same question, different prompt styles")
print("=" * 80)
compare_all_prompts("What is machine learning?")